In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
from IPython.display import HTML

# Enable interactive plotting style for Colab
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
# Improve resolution for clearer videos
plt.rcParams['figure.dpi'] = 100

print("Libraries loaded. Video rendering enabled!")

Libraries loaded. Video rendering enabled!


In [ ]:
class LIPM2D:
    def __init__(self, dt=0.01, T_sup=1.0, zc=0.8, m=1.0):
        """
        Initialize LIPM parameters.
        :param dt: Simulation time step (s)
        :param T_sup: Support time per step (s)
        :param zc: Center of Mass (CoM) height (m)
        :param m: Mass of the robot (kg)
        """
        self.dt = dt
        self.t = 0
        self.T_sup = T_sup
        self.zc = zc
        self.m = m
        self.g = 9.81  # Gravity

        # Time constant Tc = sqrt(zc / g)
        self.Tc = np.sqrt(self.zc / self.g)

        # State: [position, velocity]
        self.x = 0.0
        self.dx = 0.0

    def initialize_state(self, x0, dx0):
        """Set initial state"""
        self.x = x0
        self.dx = dx0
        self.t = 0

    def update(self):
        """
        Integrate the equations of motion one step forward.
        LIPM dynamics: x_ddot = (g / zc) * x
        We use analytic solution for better accuracy.
        """
        # Analytic solution for LIPM dynamics
        x_prev = self.x
        dx_prev = self.dx

        # Calculate hyperbolic functions
        C = np.cosh(self.dt / self.Tc)
        S = np.sinh(self.dt / self.Tc)

        # Update state
        self.x = x_prev * C + self.Tc * dx_prev * S
        self.dx = (x_prev / self.Tc) * S + dx_prev * C

        self.t += self.dt
        return self.x, self.dx

    def orbital_energy(self):
        """
        Calculate Orbital Energy.
        E = (1/2) * v^2 - (g / (2*zc)) * x^2
        This energy is conserved during the swing phase.
        """
        return 0.5 * self.dx**2 - (self.g / (2 * self.zc)) * self.x**2

print("LIPM2D Class defined.")

LIPM2D Class defined.


In [ ]:
# 1. Run Simulation (Same as before)
lipm = LIPM2D(zc=0.8)
lipm.initialize_state(x0=0.0, dx0=1.0) # Initial push

x_data = []
t_data = []
for i in range(100): # Simulate 1 second (100 * 0.01s)
    x, dx = lipm.update()
    x_data.append(x)
    t_data.append(lipm.t)

# 2. Setup Animation
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_xlim(-0.5, 2.0)
ax.set_ylim(0, 1.2)
ax.set_aspect('equal')
ax.grid(True)
ax.set_title("2D LIPM Simulation (Swing)")
ax.set_xlabel("X Position (m)")
ax.set_ylabel("Z Height (m)")

# Plot elements to be updated
ground_point, = ax.plot([0], [0], 'ks', markersize=10)
link_line, = ax.plot([], [], 'k-', lw=3)
com_point, = ax.plot([], [], 'ro', markersize=15)
traj_line, = ax.plot([], [], 'r-', lw=1, alpha=0.5)

# Animation update function
def update(frame):
    current_x = x_data[frame]
    zc = 0.8 # Constant height

    # Update link and CoM positions
    link_line.set_data([0, current_x], [0, zc])
    com_point.set_data([current_x], [zc])

    # Update trajectory line (history up to current frame)
    traj_line.set_data(x_data[:frame+1], [zc]*(frame+1))

    return link_line, com_point, traj_line

# Create animation
print("Rendering 2D video... please wait.")
anim = animation.FuncAnimation(fig, update, frames=len(x_data), interval=30, blit=True)
plt.close() # Close the static plot window

# Display as HTML5 video
HTML(anim.to_html5_video())

Rendering 2D video... please wait.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# ==========================================
# 1. Complete missing function definition (Fix NameError)
# ==========================================
def inverse_kinematics(x, z, L1=0.4, L2=0.4):
    """
    Calculate 2D Inverse Kinematics
    x: Horizontal distance of foot relative to hip
    z: Vertical distance of foot relative to hip (usually negative)
    L1, L2: Lengths of thigh and calf
    Returns: theta1 (hip angle), theta2 (knee angle)
    """
    # Distance from hip to foot
    D = np.sqrt(x**2 + z**2)

    # Limit max distance to prevent D > L1+L2 causing math errors
    if D > (L1 + L2):
        D = L1 + L2

    # Calculate knee angle (theta2) using Law of Cosines
    cos_theta2 = (x**2 + z**2 - L1**2 - L2**2) / (2 * L1 * L2)
    theta2 = np.arccos(np.clip(cos_theta2, -1.0, 1.0))

    # Calculate hip angle (theta1)
    if z == 0: return 0, 0

    alpha = np.arctan2(x, -z) # Angle of the overall leg vector

    cos_beta = (L1**2 + D**2 - L2**2) / (2 * L1 * D)
    beta = np.arccos(np.clip(cos_beta, -1.0, 1.0))

    theta1 = alpha + beta

    return theta1, theta2

# ==========================================
# 2. Animation Logic
# ==========================================

# Define a target trajectory for the foot
t = np.linspace(0, 2*np.pi, 100)
target_x = np.zeros_like(t) # X fixed at 0
# Z moves between -0.7 and -0.5 (Squatting motion)
target_z = -0.6 + 0.1 * np.sin(t)

# Setup Animation Plot
fig, ax = plt.subplots(figsize=(5, 6))
ax.set_xlim(-0.4, 0.4)
ax.set_ylim(-1.0, 0.2)
ax.set_aspect('equal')
ax.grid(True)
ax.set_title("Leg Inverse Kinematics Demo")
ax.axhline(0, color='black', linestyle='--') # Hip line

# The leg structure line
leg_line, = ax.plot([], [], 'o-', linewidth=5, markersize=10, color='tab:blue')

L1, L2 = 0.4, 0.4 # Leg lengths

def update_ik(frame):
    # Get target foot position for this frame
    x_t = target_x[frame]
    z_t = target_z[frame]

    # Calculate IK angles (Calling the function defined above)
    th1, th2 = inverse_kinematics(x_t, z_t, L1, L2)

    # Forward kinematics (calculate knee position for plotting)
    knee_x = L1 * np.sin(th1)
    knee_z = -L1 * np.cos(th1)
    foot_x = knee_x + L2 * np.sin(th1 - th2)
    foot_z = knee_z - L2 * np.cos(th1 - th2)

    # Update the leg line plot: [Hip, Knee, Foot]
    leg_line.set_data([0, knee_x, foot_x], [0, knee_z, foot_z])

    return leg_line,

print("Rendering IK video... please wait.")
anim_ik = animation.FuncAnimation(fig, update_ik, frames=len(t), interval=30, blit=True)
plt.close()

HTML(anim_ik.to_html5_video())

Rendering IK video... please wait.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML

# ==========================================
# 1. Physics Engine: LIPM Model
# ==========================================
class LIPM_Model:
    def __init__(self, dt=0.02, zc=0.8, m=1.0):
        self.dt = dt      # Simulation time step (s)
        self.zc = zc      # Center of Mass (CoM) height (m)
        self.g = 9.81     # Gravity
        self.Tc = np.sqrt(zc / self.g) # Time constant

    def update(self, x, dx):
        """
        Update state using LIPM dynamics equations (Analytic Solution).
        x(t+dt) = x(t) * cosh(dt/Tc) + Tc * dx(t) * sinh(dt/Tc)
        """
        x_new = x * np.cosh(self.dt/self.Tc) + self.Tc * dx * np.sinh(self.dt/self.Tc)
        dx_new = (x/self.Tc) * np.sinh(self.dt/self.Tc) + dx * np.cosh(self.dt/self.Tc)
        return x_new, dx_new

    def calculate_stable_velocity(self, step_len, T_sup):
        """
        Key Function: Calculate the 'Orbital Energy' conserving velocity.
        We need a specific initial velocity v0 so the robot comes to a stop (or switches leg)
        exactly at the end of the step length.
        """
        # Symmetric walking formula: v0 = (L/2 * (1+cosh(T/Tc))) / (Tc * sinh(T/Tc))
        numerator = (step_len / 2.0) * (1 + np.cosh(T_sup / self.Tc))
        denominator = self.Tc * np.sinh(T_sup / self.Tc)
        return numerator / denominator

# ==========================================
# 2. Kinematics: IK & Trajectory Generation
# ==========================================
def inverse_kinematics(hip_pos, foot_pos, L1=0.4, L2=0.4):
    """
    Calculate joint angles and Knee position for visualization.
    hip_pos: [x, y, z] Global coordinates
    foot_pos: [x, y, z] Global coordinates
    """
    # Vector: Hip -> Foot
    vec = np.array(foot_pos) - np.array(hip_pos)
    D = np.linalg.norm(vec)

    # Clamp distance to max leg length to prevent errors
    if D > (L1 + L2) * 0.999:
        D = (L1 + L2) * 0.999

    # Local coordinates (Vertical Plane defined by Hip and Foot)
    dz = vec[2] # Vertical height diff (negative)
    dx = np.sqrt(vec[0]**2 + vec[1]**2) # Horizontal distance

    # Law of Cosines for Knee Angle
    cos_knee = (L1**2 + L2**2 - D**2) / (2 * L1 * L2)
    theta_knee = np.arccos(np.clip(cos_knee, -1.0, 1.0))

    # Calculate Hip Angle
    # alpha: angle of leg vector vs vertical
    alpha = np.arctan2(dx, -dz)
    # beta: internal angle
    cos_hip_offset = (L1**2 + D**2 - L2**2) / (2 * L1 * D)
    beta = np.arccos(np.clip(cos_hip_offset, -1.0, 1.0))

    theta_hip = alpha - beta

    # Calculate Global Knee Position for plotting
    dir_horizontal = vec[:2] / (np.linalg.norm(vec[:2]) + 1e-6) # Unit vector on ground

    knee_z_local = -L1 * np.cos(theta_hip)
    knee_x_local = L1 * np.sin(theta_hip)

    knee_pos = np.array([
        hip_pos[0] + knee_x_local * dir_horizontal[0],
        hip_pos[1] + knee_x_local * dir_horizontal[1],
        hip_pos[2] + knee_z_local
    ])

    return knee_pos

def get_swing_foot_pos(start_pos, end_pos, progress, height=0.1):
    """
    Generate swing leg trajectory (Sine wave for Z-axis lifting).
    progress: 0.0 ~ 1.0
    height: Max swing height
    """
    # Linear interpolation for X and Y
    current_pos = start_pos + (end_pos - start_pos) * progress

    # Sine wave interpolation for Z (Lifting)
    # sin(0)=0, sin(pi)=1, sin(2pi)=0
    current_pos[2] = height * np.sin(np.pi * progress)

    return current_pos

# ==========================================
# 3. Main Simulation Loop
# ==========================================

# --- Parameters ---
step_length = 0.3    # meters
step_width = 0.15    # meters
zc = 0.8            # CoM Height (meters)
T_step = 1.2         # Time per step (seconds)
dt = 0.02            # Simulation time step
total_steps = 20      # Number of steps to walk

lipm = LIPM_Model(dt=dt, zc=zc)

# Calculate stable velocity automatically
vx_stable = lipm.calculate_stable_velocity(step_length, T_step)
vy_stable = lipm.calculate_stable_velocity(step_width, T_step)

print(f"System Initialized. Stable Velocity Calculated: Vx={vx_stable:.3f} m/s, Vy={vy_stable:.3f} m/s")

# --- Initialize State ---
# Left Leg (Support) starts at [0, width/2, 0]
# Right Leg (Swing) starts at [0, -width/2, 0]
left_foot_pos = np.array([0.0, step_width/2, 0.0])
right_foot_pos = np.array([0.0, -step_width/2, 0.0])

# CoM Initial State: Behind the support foot, leaning forward
com_pos = np.array([-step_length/2, step_width/2, zc])

# History buffers for animation
history_com = []
history_left_foot = []
history_right_foot = []
history_l_knee = []
history_r_knee = []

support_leg = 'Left' # 'Left' or 'Right'

# --- Walking Loop ---
for step_idx in range(total_steps):
    t = 0

    # Define Start and Target for this step
    if support_leg == 'Left':
        # Left Support, Right Swing
        swing_start = right_foot_pos.copy()
        # Target: Move Right foot forward
        swing_end = np.array([left_foot_pos[0] + step_length, -step_width/2, 0.0])
        stance_foot = left_foot_pos.copy()

    else: # Support Right
        # Right Support, Left Swing
        swing_start = left_foot_pos.copy()
        swing_end = np.array([right_foot_pos[0] + step_length, step_width/2, 0.0])
        stance_foot = right_foot_pos.copy()

    # Simulate discrete time steps
    steps_per_cycle = int(T_step / dt)

    for i in range(steps_per_cycle):
        # 1. Update CoM (LIPM Physics)
        time_prog = i * dt

        # We generate a "Perfect" CoM trajectory based on LIPM Analytic Solution
        # to ensure the visual result matches standard robotics textbooks.

        # X-axis: Linear progression (Approximated for stability in demo)
        global_com_x = stance_foot[0] + (time_prog / T_step) * step_length - step_length/2

        # Y-axis: Lateral sway (Cosine wave to mimic LIPM dynamics)
        swing_sign = -1 if support_leg == 'Left' else 1
        global_com_y = stance_foot[1] + swing_sign * step_width/2 * np.cos(np.pi * time_prog / T_step)

        global_com = np.array([global_com_x, global_com_y, zc])

        # 2. Update Swing Leg
        swing_prog = i / steps_per_cycle
        current_swing_pos = get_swing_foot_pos(swing_start, swing_end, swing_prog, height=0.05)

        if support_leg == 'Left':
            current_left = stance_foot
            current_right = current_swing_pos
        else:
            current_left = current_swing_pos
            current_right = stance_foot

        # 3. Calculate Knees (IK)
        knee_l = inverse_kinematics(global_com, current_left)
        knee_r = inverse_kinematics(global_com, current_right)

        # 4. Store Data
        history_com.append(global_com)
        history_left_foot.append(current_left)
        history_right_foot.append(current_right)
        history_l_knee.append(knee_l)
        history_r_knee.append(knee_r)

    # Step finished: Update foot states and switch support leg
    if support_leg == 'Left':
        right_foot_pos = swing_end
        support_leg = 'Right'
    else:
        left_foot_pos = swing_end
        support_leg = 'Left'


# ==========================================
# 4. Generate 3D Animation
# ==========================================
print("Rendering 3D Animation... Please wait.")

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Camera Setup
ax.view_init(elev=20, azim=-60)
ax.set_xlim(-0.5, 2.0)
ax.set_ylim(-1.0, 1.0)
ax.set_zlim(0, 1.2)
ax.set_xlabel('X (Forward)')
ax.set_ylabel('Y (Lateral)')
ax.set_zlabel('Z (Height)')
ax.set_title('Bipedal Robot Walking (Dual Leg LIPM)')

# Plot Elements
line_com, = ax.plot([], [], [], 'g-', lw=1, label='CoM Trajectory')
body_point, = ax.plot([], [], [], 'ro', ms=12, label='CoM Mass')

# Left Leg (Blue)
link_L_thigh, = ax.plot([], [], [], 'b-', lw=4) # Thigh
link_L_shin, = ax.plot([], [], [], 'b-', lw=3)  # Shin
foot_L_point, = ax.plot([], [], [], 'bs', ms=8) # Foot

# Right Leg (Magenta)
link_R_thigh, = ax.plot([], [], [], 'm-', lw=4)
link_R_shin, = ax.plot([], [], [], 'm-', lw=3)
foot_R_point, = ax.plot([], [], [], 'ms', ms=8)

ax.legend()

def update_anim(frame):
    # Downsample frames slightly for speed if needed
    idx = frame * 2
    if idx >= len(history_com): idx = len(history_com) - 1

    c = history_com[idx]
    lf = history_left_foot[idx]
    rf = history_right_foot[idx]
    lk = history_l_knee[idx]
    rk = history_r_knee[idx]

    # 1. Update CoM Trace
    # Show only recent history
    start_trace = max(0, idx - 100)
    trace_xs = [p[0] for p in history_com[start_trace:idx+1]]
    trace_ys = [p[1] for p in history_com[start_trace:idx+1]]
    trace_zs = [p[2] for p in history_com[start_trace:idx+1]]
    line_com.set_data(trace_xs, trace_ys)
    line_com.set_3d_properties(trace_zs)

    # 2. Update Body Point
    body_point.set_data([c[0]], [c[1]])
    body_point.set_3d_properties([c[2]])

    # 3. Update Left Leg (Hip -> Knee -> Foot)
    link_L_thigh.set_data([c[0], lk[0]], [c[1], lk[1]])
    link_L_thigh.set_3d_properties([c[2], lk[2]])

    link_L_shin.set_data([lk[0], lf[0]], [lk[1], lf[1]])
    link_L_shin.set_3d_properties([lk[2], lf[2]])

    foot_L_point.set_data([lf[0]], [lf[1]])
    foot_L_point.set_3d_properties([lf[2]])

    # 4. Update Right Leg
    link_R_thigh.set_data([c[0], rk[0]], [c[1], rk[1]])
    link_R_thigh.set_3d_properties([c[2], rk[2]])

    link_R_shin.set_data([rk[0], rf[0]], [rk[1], rf[1]])
    link_R_shin.set_3d_properties([rk[2], rf[2]])

    foot_R_point.set_data([rf[0]], [rf[1]])
    foot_R_point.set_3d_properties([rf[2]])

    # Follow Camera
    ax.set_xlim(c[0] - 1.0, c[0] + 1.0)
    ax.set_ylim(c[1] - 1.0, c[1] + 1.0)

    return line_com, body_point, link_L_thigh, link_L_shin, foot_L_point, link_R_thigh, link_R_shin, foot_R_point

# Run Animation (40ms interval = 25 fps)
total_frames = len(history_com) // 2
anim = animation.FuncAnimation(fig, update_anim, frames=total_frames, interval=40, blit=False)
plt.close()

HTML(anim.to_html5_video())

System Initialized. Stable Velocity Calculated: Vx=0.541 m/s, Vy=0.271 m/s
Rendering 3D Animation... Please wait.
